In [1]:
import cudf
import pyarrow as pa
import pandas as pd
import numpy as np
import cupy as cp

In [2]:
df = cudf.read_parquet('../data/all_states_us_lat_long.parquet/*').reset_index().drop(['index'])

In [3]:
# df['block_id'] = (df.GISJOIN%10000).astype('int32')
# df['State'] = (df.GISJOIN/1000_000_000_000_000).astype('int32')
df['GISJOIN_og'] = df.GISJOIN
df['GISJOIN'] = (df.GISJOIN/1000).astype('int')

In [4]:
df

,GISJOIN,x,y,p_id,GISJOIN_og
0,1000100211002,-9.662807e+06,3.831871e+06,0,1000100211002002
1,1000100211002,-9.663283e+06,3.832983e+06,1,1000100211002002
2,1000100211002,-9.663042e+06,3.832885e+06,2,1000100211002002
3,1000100211002,-9.662848e+06,3.833558e+06,3,1000100211002002
4,1000100211002,-9.662436e+06,3.833098e+06,4,1000100211002002
...,...,...,...,...,...
308745533,56004509511002,-1.162458e+07,5.461661e+06,308745533,56004509511002304
308745534,56004509511002,-1.162415e+07,5.461604e+06,308745534,56004509511002304
308745535,56004509511002,-1.162405e+07,5.461376e+06,308745535,56004509511002304
308745536,56004509511002,-1.162429e+07,5.461773e+06,308745536,56004509511002304


## Adding gender block wise

In [5]:
df_cat = cudf.read_csv('/home/ajay/data/census/all_states_block_group/nhgis0016_csv/nhgis0016_ds176_20105_2010_blck_grp.csv')

df_cat.STATEA = df_cat.STATEA.astype('int')
df_cat = df_cat.query('STATEA != 72')

df_cat.GISJOIN = df_cat.GISJOIN.str.replace('G', '').astype('int')

# sex_by_age_cols = ['JLZE001','JLZE002','JLZE003','JLZE004','JLZE005','JLZE006','JLZE007','JLZE008','JLZE009','JLZE010','JLZE011','JLZE012','JLZE013','JLZE014','JLZE015','JLZE016','JLZE017','JLZE018','JLZE019','JLZE020','JLZE021','JLZE022','JLZE023','JLZE024','JLZE025','JLZE026','JLZE027','JLZE028','JLZE029','JLZE030','JLZE031','JLZE032','JLZE033','JLZE034','JLZE035','JLZE036','JLZE037','JLZE038','JLZE039','JLZE040','JLZE041','JLZE042','JLZE043','JLZE044','JLZE045','JLZE046','JLZE047','JLZE048','JLZE049']
sex_by_age_cols = ['JLZE001', 'JLZE002', 'JLZE026']
df_cat = df_cat[['GISJOIN']+sex_by_age_cols]

for i in df_cat.columns:
    if i != 'GISJOIN':
        df_cat[i] = df_cat[i].astype('int32')

In [6]:
df_cat.JLZE001.value_counts()

0        1125
800       197
1000      197
1135      196
931       193
         ... 
18137       1
19974       1
22129       1
28320       1
39248       1
Name: JLZE001, Length: 5387, dtype: int32

In [7]:
df_cat.JLZE001.value_counts()

0        1125
800       197
1000      197
1135      196
931       193
         ... 
18137       1
19974       1
22129       1
28320       1
39248       1
Name: JLZE001, Length: 5387, dtype: int32

In [8]:
df_cat.JLZE002 = df_cat.JLZE002/df_cat.JLZE001
df_cat.JLZE026 = df_cat.JLZE026/df_cat.JLZE001

In [9]:
df_cat = df_cat.nans_to_nulls()

In [10]:
df_cat = df_cat.fillna(0)

In [11]:
df_cat.JLZE002 = df_cat.JLZE002.fillna(0)
df_cat.JLZE026 = df_cat.JLZE026.fillna(0)

In [12]:
df_cat.JLZE026.unique()

0         0.000000
1         0.000826
2         0.000840
3         0.000842
4         0.000866
            ...   
132301    0.983698
132302    0.985891
132303    0.998169
132304    0.998315
132305    1.000000
Name: JLZE026, Length: 132306, dtype: float64

In [13]:
df_cat.JLZE002.unique()

0         0.000000
1         0.001685
2         0.001831
3         0.014109
4         0.016302
            ...   
132301    0.999134
132302    0.999158
132303    0.999160
132304    0.999174
132305    1.000000
Name: JLZE002, Length: 132306, dtype: float64

In [14]:
df_cat.drop_column('JLZE001')
df_cat

,GISJOIN,JLZE002,JLZE026
0,1000100201001,0.427788,0.572212
1,1000100201002,0.526149,0.473851
2,1000100202001,0.393755,0.606245
3,1000100202002,0.516266,0.483734
4,1000100203001,0.431150,0.568850
...,...,...,...
217735,56004509511001,0.541538,0.458462
217736,56004509511002,0.576900,0.423100
217737,56004509513001,0.472464,0.527536
217738,56004509513002,0.456000,0.544000


In [15]:
temp = df.merge(df_cat, on='GISJOIN')

In [16]:
del(df)
del(df_cat)

In [17]:
temp.drop_column('GISJOIN')

In [18]:
# temp.index = temp.GISJOIN_og
temp.rename({'GISJOIN_og':'GISJOIN'}, inplace=True)
temp

,x,y,p_id,GISJOIN,JLZE002,JLZE026
0,-9.786339e+06,3.570557e+06,5248,1000300112024031,0.500792,0.499208
1,-9.786446e+06,3.570563e+06,5249,1000300112024031,0.500792,0.499208
2,-9.786273e+06,3.570724e+06,5250,1000300112024031,0.500792,0.499208
3,-9.786256e+06,3.570404e+06,5251,1000300112024031,0.500792,0.499208
4,-9.786152e+06,3.570634e+06,5252,1000300112024031,0.500792,0.499208
...,...,...,...,...,...,...
308745304,-1.201720e+07,5.467818e+06,308735867,56004300003013056,0.445183,0.554817
308745305,-1.201725e+07,5.467802e+06,308735868,56004300003013056,0.445183,0.554817
308745306,-1.201717e+07,5.467729e+06,308735869,56004300003013056,0.445183,0.554817
308745307,-1.201720e+07,5.467799e+06,308735870,56004300003013056,0.445183,0.554817


In [ ]:
temp.to_parquet('temp.parquet')

/home/ajay/anaconda3/envs/rapids_prod_0.12/lib/python3.6/site-packages/cudf/io/parquet.py:70: UserWarning: Using CPU via PyArrow to write Parquet dataset, this will be GPU accelerated in the future
  "Using CPU via PyArrow to write Parquet dataset, this will "
